In [ ]:
# Install the Kaggle library
%pip install kaggle

# Import necessary libraries
import os
from google.colab import userdata

# Get the Kaggle API key from Colab secrets
kaggle_secret = userdata.get('KAGGLE_KEY')

# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Write the kaggle.json file with the secret content
# Use a temporary file to avoid issues with direct writing
with open('/tmp/kaggle.json', 'w') as f:
    f.write(kaggle_secret)

# Move the temporary file to the correct location
!mv /tmp/kaggle.json ~/.kaggle/kaggle.json

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API credentials set up successfully.")

Kaggle API credentials set up successfully.


In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "gm_games_2022.csv" # Replace with the actual file path

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "farhadzamani/chess-com-2022",
  file_path,
)

print("First 5 records:", df.head())

/tmp/ipython-input-2931988893.py:9: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 474M/474M [00:07<00:00, 65.0MB/s]

Extracting zip of gm_games_2022.csv...



/usr/local/lib/python3.12/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


First 5 records:    Unnamed: 0                                      game_id  \
0           0  https://www.chess.com/game/live/34762832595   
1           1  https://www.chess.com/game/live/34842040547   
2           2  https://www.chess.com/game/live/34845639495   
3           3  https://www.chess.com/game/live/34849246827   
4           4  https://www.chess.com/game/live/52834844739   

                                      game_url  \
0  https://www.chess.com/game/live/34762832595   
1  https://www.chess.com/game/live/34842040547   
2  https://www.chess.com/game/live/34845639495   
3  https://www.chess.com/game/live/34849246827   
4  https://www.chess.com/game/live/52834844739   

                                                 pgn time_control  \
0  [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...        900+5   
1  [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...        900+5   
2  [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...        900+5   
3  [Event "Live Chess"]\n[Site "C

In [ ]:
!pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=0677d1ecd19ef2d632ff1a873a677c4f80c93f827d722449db097267afa93d53
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [ ]:
filtered_df = df[
    (df['rules'] == 'chess') &
    (df['time_class'] == 'blitz') &
    (df['pgn'].notnull())
].copy()

print(f"Original dataset size: {len(df):,}")
print(f"After filtering: {len(filtered_df):,}")
print(f"Percentage remaining: {len(filtered_df)/len(df)*100:.1f}%")

Original dataset size: 754,429
After filtering: 498,479
Percentage remaining: 66.1%


In [ ]:
import chess
import chess.pgn
from io import StringIO

def pgn_to_fens(pgn_string, max_positions=5, skip_early_moves=10, max_moves=80):
    """
    Convert a PGN string to a list of FEN positions

    Args:
        pgn_string: The PGN game string
        max_positions: Maximum FEN positions to extract per game
        skip_early_moves: Skip this many moves from the beginning
        max_moves: Stop extracting after this many moves

    Returns:
        List of FEN strings
    """
    try:
        # Parse the PGN
        pgn_io = StringIO(pgn_string)
        game = chess.pgn.read_game(pgn_io)

        if game is None:
            return []

        board = game.board()
        fens = []
        move_count = 0

        # Go through all moves
        for move in game.mainline_moves():
            move_count += 1
            board.push(move)

            # Skip early moves and stop after max_moves
            if move_count < skip_early_moves or move_count > max_moves:
                continue

            # Sample positions at regular intervals
            if len(fens) < max_positions:
                # Take every Nth move to spread positions across the game
                interval = max(1, (max_moves - skip_early_moves) // max_positions)
                if (move_count - skip_early_moves) % interval == 0:
                    fens.append(board.fen())

        return fens

    except Exception as e:
        print(f"Error processing PGN: {e}")
        return []

# Test with first PGN
test_pgn = filtered_df.iloc[0]['pgn']
test_fens = pgn_to_fens(test_pgn)
print(f"Extracted {len(test_fens)} FENs from first game:")
for i, fen in enumerate(test_fens):
    print(f"Position {i+1}: {fen}")

Extracted 5 FENs from first game:
Position 1: r1bqkb1r/pp3ppp/2n1pn2/1Bpp4/8/1P2PN2/PBPP1PPP/RN1QK2R w KQkq - 0 6
Position 2: r3k1r1/p1qb1p1p/2pb4/2ppp3/5P2/1P2P3/PBPP2PP/RN1Q1RK1 w q - 0 13
Position 3: 2kr4/p4p1p/2p1b3/2pp3q/5Qr1/1PN1P3/P1PP1RPP/5RK1 w - - 10 20
Position 4: 6r1/p3Qp2/k1p1b3/3p3p/4qNr1/1P2P3/P2P1RPP/5RK1 w - - 3 27
Position 5: 6r1/p4p2/1kp5/3p4/P2PbNrp/4P3/P4RPP/2R3K1 w - - 1 34


In [ ]:
import random
from tqdm import tqdm
import pandas as pd

def create_fen_dataset(df, target_fens=1000000, max_games=None):
    """
    Create a dataset of FEN positions from PGN games
    """
    print(f"Creating FEN dataset from {len(df)} games...")
    print(f"Target: {target_fens:,} FEN positions")

    all_fens = []
    games_processed = 0
    games_with_fens = 0

    # Shuffle the dataframe to get random games
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Process games until we reach target
    for idx, row in tqdm(df_shuffled.iterrows(), total=len(df_shuffled), desc="Processing games"):
        if len(all_fens) >= target_fens:
            break

        if max_games and games_processed >= max_games:
            break

        pgn_string = row['pgn']
        fens = pgn_to_fens(pgn_string, max_positions=3, skip_early_moves=8, max_moves=70)

        if fens:
            all_fens.extend(fens)
            games_with_fens += 1

        games_processed += 1

        # Progress update every 10k games
        if games_processed % 10000 == 0:
            print(f"Processed {games_processed:,} games, collected {len(all_fens):,} FENs")

    print(f"\nDataset creation complete!")
    print(f"Games processed: {games_processed:,}")
    print(f"Games with valid FENs: {games_with_fens:,}")
    print(f"Total FENs collected: {len(all_fens):,}")

    return all_fens

# Create the FEN dataset (this will take a while!)
fen_list = create_fen_dataset(filtered_df, target_fens=1000000)

Creating FEN dataset from 498479 games...
Target: 1,000,000 FEN positions


Processing games:   2%|▏         | 10028/498479 [00:52<57:43, 141.04it/s]

Processed 10,000 games, collected 28,462 FENs


Processing games:   4%|▍         | 20046/498479 [01:44<36:20, 219.44it/s]

Processed 20,000 games, collected 56,889 FENs


...

Processed 340,000 games, collected 968,275 FENs


Processing games:  70%|███████   | 350029/498479 [31:01<17:07, 144.53it/s]

Processed 350,000 games, collected 996,696 FENs


Processing games:  70%|███████   | 351158/498479 [31:08<13:03, 187.96it/s]


Dataset creation complete!
Games processed: 351,158
Games with valid FENs: 349,881
Total FENs collected: 1,000,002


In [ ]:
import torch
PIECE_TO_INT = {
    'p': 1, 'n': 2, 'b': 3, 'r': 4, 'q': 5, 'k': 6,
    'P': 7, 'N': 8, 'B': 9, 'R': 10, 'Q': 11, 'K': 12
}
INT_TO_PIECE = {v: k for k, v in PIECE_TO_INT.items()}
ABSORBING_STATE_INT = 13
VOCAB_SIZE = 14
def fen_to_tensor(fen_string: str) -> torch.Tensor:
    """
    Parses a FEN string and converts it into a 1D tensor of 64 tokens.
    Each token represents a square on the board, with an integer value
    corresponding to the piece on it (0 for empty).

    Args:
        fen_string (str): The FEN string for a board position.

    Returns:
        torch.Tensor: A 1D tensor of shape (64,) with integer piece representations.
    """
    # The board tensor, initialized to 0 (empty)
    board_tensor = torch.zeros(64, dtype=torch.long)

    # The first part of FEN is the piece placement
    piece_placement = fen_string.split(' ')[0]

    rank_index = 7  # Start from rank 8 (index 7)
    file_index = 0  # Start from file 'a' (index 0)

    for char in piece_placement:
        if char == '/':
            rank_index -= 1
            file_index = 0
        elif char.isdigit():
            file_index += int(char)
        else:
            square_index = rank_index * 8 + file_index
            board_tensor[square_index] = PIECE_TO_INT[char]
            file_index += 1

    return board_tensor

def tensor_to_fen(board_tensor: torch.Tensor, active_color: str ='w', castling: str ='KQkq', en_passant: str ='-', halfmove_clock: int =0, fullmove_number: int=1) -> str:
    """
    Converts a 1D tensor of 64 tokens back into a FEN string.

    Note: This function only reconstructs the piece placement part of the FEN.
    Other game state information (turn, castling, etc.) must be provided.

    Args:
        board_tensor (torch.Tensor): A 1D tensor of shape (64,) representing the board.
        active_color (str): 'w' or 'b'.
        castling (str): Castling availability (e.g., 'KQkq', '-', 'Kq').
        en_passant (str): En passant target square (e.g., 'e3', '-').
        halfmove_clock (int): Halfmove clock value.
        fullmove_number (int): Fullmove number.

    Returns:
        str: The full FEN string for the position.
    """
    if board_tensor.shape != (64,):
        raise ValueError("Input tensor must have shape (64,)")

    fen_parts = []
    for rank_index in range(7, -1, -1):  # Iterate from rank 8 down to 1
        rank_fen = ""
        empty_count = 0
        for file_index in range(8):  # Iterate from file 'a' to 'h'
            square_index = rank_index * 8 + file_index
            piece_int = board_tensor[square_index].item()

            if piece_int == 0:
                empty_count += 1
            else:
                if empty_count > 0:
                    rank_fen += str(empty_count)
                    empty_count = 0
                rank_fen += INT_TO_PIECE[piece_int]

        if empty_count > 0:
            rank_fen += str(empty_count)

        fen_parts.append(rank_fen)

    piece_placement = "/".join(fen_parts)

    # Combine all parts of the FEN string
    full_fen = f"{piece_placement} {active_color} {castling} {en_passant} {halfmove_clock} {fullmove_number}"

    return full_fen

In [ ]:
def create_chess_tensor_dataset(fen_list, output_file='chess_positions_gm.pt'):
    """
    Convert FEN list to tensors and save
    """
    print(f"Converting {len(fen_list):,} FENs to tensors...")

    board_tensors = []
    failed_conversions = 0

    for i, fen_string in enumerate(tqdm(fen_list, desc="Converting FENs")):
        try:
            # Extract just the board position part (before first space)
            board_fen = fen_string.split()[0] + " w KQkq - 0 1"  # Standardize the rest
            board_tensor = fen_to_tensor(board_fen)
            board_tensors.append(board_tensor)
        except Exception as e:
            failed_conversions += 1
            if failed_conversions < 10:  # Only print first few errors
                print(f"Failed to convert FEN {i}: {fen_string[:50]}... Error: {e}")

    print(f"Successfully converted: {len(board_tensors):,}")
    print(f"Failed conversions: {failed_conversions:,}")

    # Stack all tensors
    final_dataset_tensor = torch.stack(board_tensors).long()
    print(f"Final tensor shape: {final_dataset_tensor.shape}")

    # Save the dataset
    torch.save(final_dataset_tensor, output_file)
    print(f"Dataset saved to '{output_file}'")

    return final_dataset_tensor

# Convert to tensors and save
dataset_tensor = create_chess_tensor_dataset(fen_list)

Converting 1,000,002 FENs to tensors...


Converting FENs: 100%|██████████| 1000002/1000002 [02:56<00:00, 5650.50it/s]


Successfully converted: 1,000,002
Failed conversions: 0
Final tensor shape: torch.Size([1000002, 64])
Dataset saved to 'chess_positions_gm.pt'


In [ ]:
# Load and verify the saved dataset
print("Loading saved dataset...")
loaded_tensor = torch.load('chess_positions_gm.pt')
print(f"Loaded tensor shape: {loaded_tensor.shape}")
print(f"Data type: {loaded_tensor.dtype}")
print(f"Value range: {loaded_tensor.min().item()} to {loaded_tensor.max().item()}")
print(f"Vocabulary size: {VOCAB_SIZE}")

# Test a few positions
print("\nTesting first 3 positions:")
for i in range(3):
    board_tensor = loaded_tensor[i]
    reconstructed_fen = tensor_to_fen(board_tensor)
    print(f"Position {i+1}: {reconstructed_fen}")

# Check for any values outside expected range
invalid_values = (loaded_tensor < 0) | (loaded_tensor >= VOCAB_SIZE)
if invalid_values.any():
    print(f"Warning: Found {invalid_values.sum().item()} invalid values!")
else:
    print("✓ All values are within valid range [0, 13]")

print(f"\n✓ Dataset ready! Shape: {loaded_tensor.shape}")
print("You can now use this dataset with your existing training pipeline.")

Loading saved dataset...
Loaded tensor shape: torch.Size([1000002, 64])
Data type: torch.int64
Value range: 0 to 12
Vocabulary size: 14

Testing first 3 positions:
Position 1: r1bqkb1r/pppp1pp1/2n2n1p/4p3/2B1P3/3P1N2/PPP2PPP/RNBQK2R w KQkq - 0 1
Position 2: r2q2k1/ppp2pb1/5npp/3P1b2/1n1P4/1B3N1P/PP1N1PP1/R1B1Q1K1 w KQkq - 0 1
Position 3: 6k1/2Q2pb1/r4npp/pp1q4/2pP4/4BN1P/PP3PP1/R5K1 w KQkq - 0 1
✓ All values are within valid range [0, 13]

✓ Dataset ready! Shape: torch.Size([1000002, 64])
You can now use this dataset with your existing training pipeline.


In [ ]:
# Copy the generated dataset file to your Google Drive
!cp chess_positions_gm.pt /content/drive/MyDrive/chess_positions_gm.pt

print("File copied to Google Drive.")

File copied to Google Drive.
